In [23]:
import pandas as pd
import plotly.express as px

# Load dataset
df = pd.read_csv("GuttmacherInstituteAbortionDataByState.csv")

# Clean columns
df.columns = df.columns.str.strip().str.replace('–', '-').str.replace('—', '-').str.replace(' ', '_') \
    .str.replace(',', '').str.replace('(', '').str.replace(')', '').str.replace('’', '').str.replace('.', '_')

df.rename(columns={'U.S._State': 'U_S__State'}, inplace=True)

# Convert columns
df['No__of_abortion_clinics_2020'] = pd.to_numeric(df['No__of_abortion_clinics_2020'], errors='coerce')
df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'] = pd.to_numeric(
    df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'], errors='coerce')
df['No__of_abortions_by_state_of_occurrence_2020'] = pd.to_numeric(
    df['No__of_abortions_by_state_of_occurrence_2020'], errors='coerce')
df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020'] = pd.to_numeric(
    df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020'], errors='coerce')

# Estimate population of women aged 15–44
df['Pop_15_44'] = (df['No__of_abortions_by_state_of_occurrence_2020'] * 1000) / df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020']

# Add state codes
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
df['State_Code'] = df['U_S__State'].map(state_abbreviations)
df = df.dropna(subset=['State_Code', 'Pop_15_44'])

# Add land area (square miles)
land_area = {
    'AL': 52420, 'AK': 665384, 'AZ': 113990, 'AR': 53179, 'CA': 163695, 'CO': 104094,
    'CT': 5543, 'DE': 2489, 'DC': 68, 'FL': 65758, 'GA': 59425, 'HI': 10932,
    'ID': 83569, 'IL': 57914, 'IN': 36420, 'IA': 56273, 'KS': 82278, 'KY': 40408,
    'LA': 52378, 'ME': 35380, 'MD': 12406, 'MA': 10554, 'MI': 96714, 'MN': 86936,
    'MS': 48432, 'MO': 69707, 'MT': 147040, 'NE': 77348, 'NV': 110572, 'NH': 9349,
    'NJ': 8723, 'NM': 121590, 'NY': 54555, 'NC': 53819, 'ND': 70698, 'OH': 44826,
    'OK': 69899, 'OR': 98379, 'PA': 46054, 'RI': 1545, 'SC': 32020, 'SD': 77116,
    'TN': 42144, 'TX': 268596, 'UT': 84897, 'VT': 9616, 'VA': 42775, 'WA': 71298,
    'WV': 24230, 'WI': 65496, 'WY': 97813
}
df['Land_Area'] = df['State_Code'].map(land_area)
df = df.dropna(subset=['Land_Area'])

# Normalize clinics and funding
df['Clinics_Score'] = df['No__of_abortion_clinics_2020'] / df['No__of_abortion_clinics_2020'].max()
df['Funding_Score'] = df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'] / df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'].max()
df['Raw_Resource_Score'] = df['Clinics_Score'] + df['Funding_Score']

# Adjusted score per land area
df['Adjusted_Resource_Score'] = df['Raw_Resource_Score'] / df['Land_Area']

# --------------------
# Plot 1: Raw Score (Unadjusted)
# --------------------
fig = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Raw_Resource_Score',
    color_continuous_scale=px.colors.sequential.Blues,
    scope='usa',
    title='California is Unmatched in Abortion Resources<br><sup>Total Clinics and State Abortion Funding — Without Adjusting for Population or Geography</sup>'
)
fig.update_layout(
    template="plotly_white",
    coloraxis_colorbar_title="Total Resource Score<br>(Clinics + Funding, Normalized)",
    width=1150,
    height=550,
    margin=dict(l=60, r=60, t=90, b=40),
    title_x=0.5,
    title_font_size=28
)
fig.show()

# --------------------
# Plot 2: Adjusted Score (Per Square Mile, Normalized Resources)
# --------------------
labels = ["0-20%", "20–40%", "40–60%", "60–80%", "80-100%"]
df['Density_Bin_Label'] = pd.qcut(df['Adjusted_Resource_Score'], q=5, labels=labels)
df['Density_Bin_Label'] = pd.Categorical(df['Density_Bin_Label'], categories=labels, ordered=True)

color_scale_discrete = ['#deebf7', '#9ecae1', '#6baed6', '#3182bd', '#08519c']

fig = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Density_Bin_Label',
    color_discrete_sequence=color_scale_discrete,
    category_orders={'Density_Bin_Label': labels},
    scope='usa',
    title='Why the East Coast Dominates in Abortion Access<br><sup>Based on Normalized Clinics and State Funding per Square Mile</sup>'
)
fig.update_layout(
    template="plotly_white",
    legend_title="Percentile Based on:<br>(Normalized Clinics + Funding) / Land Area",
    width=1150,
    height=550,
    margin=dict(l=60, r=60, t=90, b=40),
    title_x=0.5,
    title_font_size=28
)
fig.show()


In [24]:
import pandas as pd
import plotly.express as px

# Load dataset
df = pd.read_csv("GuttmacherInstituteAbortionDataByState.csv")

# Clean columns
df.columns = df.columns.str.strip().str.replace('–', '-').str.replace('—', '-').str.replace(' ', '_') \
    .str.replace(',', '').str.replace('(', '').str.replace(')', '').str.replace('’', '').str.replace('.', '_')

df.rename(columns={'U.S._State': 'U_S__State'}, inplace=True)

# Convert columns
df['No__of_abortion_clinics_2020'] = pd.to_numeric(df['No__of_abortion_clinics_2020'], errors='coerce')
df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'] = pd.to_numeric(
    df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'], errors='coerce')
df['No__of_abortions_by_state_of_occurrence_2020'] = pd.to_numeric(
    df['No__of_abortions_by_state_of_occurrence_2020'], errors='coerce')
df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020'] = pd.to_numeric(
    df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020'], errors='coerce')

# Estimate population of women aged 15–44
df['Pop_15_44'] = (df['No__of_abortions_by_state_of_occurrence_2020'] * 1000) / df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020']

# Add state codes
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
df['State_Code'] = df['U_S__State'].map(state_abbreviations)
df = df.dropna(subset=['State_Code', 'Pop_15_44'])

# Normalize clinics and funding
df['Clinics_Score'] = df['No__of_abortion_clinics_2020'] / df['No__of_abortion_clinics_2020'].max()
df['Funding_Score'] = df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'] / df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'].max()
df['Raw_Resource_Score'] = df['Clinics_Score'] + df['Funding_Score']

# ✅ Adjust by population (per capita)
df['Adjusted_Resource_Score'] = df['Raw_Resource_Score'] / df['Pop_15_44']

# --------------------
# Plot 1: Raw Score (Unadjusted)
# --------------------
fig = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Raw_Resource_Score',
    color_continuous_scale=px.colors.sequential.Blues,
    scope='usa',
    title='California is Unmatched in Abortion Resources<br><sup>Total Clinics and State Abortion Funding — Without Adjusting for Population or Geography</sup>'
)
fig.update_layout(
    template="plotly_white",
    coloraxis_colorbar_title="Total Resource Score<br>(Clinics + Funding, Normalized)",
    width=1150,
    height=550,
    margin=dict(l=60, r=60, t=90, b=40),
    title_x=0.5,
    title_font_size=20
)
fig.show()

# --------------------
# Plot 2: Adjusted Score (Per Capita, Normalized Resources)
# --------------------
labels = ["Bottom 20%", "20–40%", "40–60%", "60–80%", "Top 20%"]
df['Density_Bin_Label'] = pd.qcut(df['Adjusted_Resource_Score'], q=5, labels=labels)
df['Density_Bin_Label'] = pd.Categorical(df['Density_Bin_Label'], categories=labels, ordered=True)

color_scale_discrete = ['#deebf7', '#9ecae1', '#6baed6', '#3182bd', '#08519c']

fig = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Density_Bin_Label',
    color_discrete_sequence=color_scale_discrete,
    category_orders={'Density_Bin_Label': labels},
    scope='usa',
    title='Why the East Coast Dominates in Abortion Access<br><sup>Based on Normalized Clinics and State Funding per Capita</sup>'
)
fig.update_layout(
    template="plotly_white",
    legend_title="Percentile Based on:<br>(Normalized Clinics + Funding) / Population",
    width=1150,
    height=550,
    margin=dict(l=60, r=60, t=90, b=40),
    title_x=0.5,
    title_font_size=20
)
fig.show()


In [26]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load dataset
df = pd.read_csv("GuttmacherInstituteAbortionDataByState.csv")

# Clean columns
df.columns = df.columns.str.strip().str.replace('–', '-').str.replace('—', '-').str.replace(' ', '_') \
    .str.replace(',', '').str.replace('(', '').str.replace(')', '').str.replace('’', '').str.replace('.', '_')

df.rename(columns={'U.S._State': 'U_S__State'}, inplace=True)

# Convert columns
df['No__of_abortion_clinics_2020'] = pd.to_numeric(df['No__of_abortion_clinics_2020'], errors='coerce')
df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'] = pd.to_numeric(
    df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'], errors='coerce')
df['No__of_abortions_by_state_of_occurrence_2020'] = pd.to_numeric(
    df['No__of_abortions_by_state_of_occurrence_2020'], errors='coerce')
df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020'] = pd.to_numeric(
    df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020'], errors='coerce')

# Estimate population of women aged 15–44
df['Pop_15_44'] = (df['No__of_abortions_by_state_of_occurrence_2020'] * 1000) / df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020']

# Add state codes
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
df['State_Code'] = df['U_S__State'].map(state_abbreviations)

# Add land area (square miles)
land_area = {
    'AL': 52420, 'AK': 665384, 'AZ': 113990, 'AR': 53179, 'CA': 163695, 'CO': 104094,
    'CT': 5543, 'DE': 2489, 'DC': 68, 'FL': 65758, 'GA': 59425, 'HI': 10932,
    'ID': 83569, 'IL': 57914, 'IN': 36420, 'IA': 56273, 'KS': 82278, 'KY': 40408,
    'LA': 52378, 'ME': 35380, 'MD': 12406, 'MA': 10554, 'MI': 96714, 'MN': 86936,
    'MS': 48432, 'MO': 69707, 'MT': 147040, 'NE': 77348, 'NV': 110572, 'NH': 9349,
    'NJ': 8723, 'NM': 121590, 'NY': 54555, 'NC': 53819, 'ND': 70698, 'OH': 44826,
    'OK': 69899, 'OR': 98379, 'PA': 46054, 'RI': 1545, 'SC': 32020, 'SD': 77116,
    'TN': 42144, 'TX': 268596, 'UT': 84897, 'VT': 9616, 'VA': 42775, 'WA': 71298,
    'WV': 24230, 'WI': 65496, 'WY': 97813
}
df['Land_Area'] = df['State_Code'].map(land_area)

# Drop incomplete rows
df = df.dropna(subset=['State_Code', 'Pop_15_44', 'Land_Area'])

# Normalize clinics and funding
df['Clinics_Score'] = df['No__of_abortion_clinics_2020'] / df['No__of_abortion_clinics_2020'].max()
df['Funding_Score'] = df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'] / df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'].max()
df['Raw_Resource_Score'] = df['Clinics_Score'] + df['Funding_Score']

# --------------------
# Plot 2A: Adjusted by Population (Per Capita)
# --------------------
df['Adj_Pop'] = df['Raw_Resource_Score'] / df['Pop_15_44']
labels = ["Bottom 20%", "20–40%", "40–60%", "60–80%", "Top 20%"]
df['Bin_Pop'] = pd.qcut(df['Adj_Pop'], q=5, labels=labels)
df['Bin_Pop'] = pd.Categorical(df['Bin_Pop'], categories=labels, ordered=True)

fig1 = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Bin_Pop',
    color_discrete_sequence=['#deebf7', '#9ecae1', '#6baed6', '#3182bd', '#08519c'],
    category_orders={'Bin_Pop': labels},
    scope='usa',
    title='Access by Normalized Clinics and Funding per Capita'
)
fig1.update_layout(
    legend_title="Percentile:<br>(Normalized Clinics + Funding) / Population",
    width=1000,
    height=550,
    template="plotly_white",
    title_x=0.5,
    title_font_size=20
)

# --------------------
# Plot 2B: Adjusted by Land Area (Per Square Mile)
# --------------------
df['Adj_Area'] = df['Raw_Resource_Score'] / df['Land_Area']
df['Bin_Area'] = pd.qcut(df['Adj_Area'], q=5, labels=labels)
df['Bin_Area'] = pd.Categorical(df['Bin_Area'], categories=labels, ordered=True)

fig2 = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Bin_Area',
    color_discrete_sequence=['#deebf7', '#9ecae1', '#6baed6', '#3182bd', '#08519c'],
    category_orders={'Bin_Area': labels},
    scope='usa',
    title='Access by Normalized Clinics and Funding per Square Mile'
)
fig2.update_layout(
    legend_title="Percentile:<br>(Normalized Clinics + Funding) / Land Area",
    width=1000,
    height=550,
    template="plotly_white",
    title_x=0.5,
    title_font_size=20
)

# Show both plots (side-by-side if in notebook / interactive mode)
fig1.show()
fig2.show()


In [31]:
import pandas as pd
import plotly.express as px
import numpy as np

# Load dataset
df = pd.read_csv("GuttmacherInstituteAbortionDataByState.csv")

# Clean columns
df.columns = df.columns.str.strip().str.replace('–', '-').str.replace('—', '-').str.replace(' ', '_') \
    .str.replace(',', '').str.replace('(', '').str.replace(')', '').str.replace('’', '').str.replace('.', '_')

df.rename(columns={'U.S._State': 'U_S__State'}, inplace=True)

# Convert columns
df['No__of_abortion_clinics_2020'] = pd.to_numeric(df['No__of_abortion_clinics_2020'], errors='coerce')
df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'] = pd.to_numeric(
    df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'], errors='coerce')
df['No__of_abortions_by_state_of_occurrence_2020'] = pd.to_numeric(
    df['No__of_abortions_by_state_of_occurrence_2020'], errors='coerce')
df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020'] = pd.to_numeric(
    df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020'], errors='coerce')

# Estimate population of women aged 15–44
df['Pop_15_44'] = (df['No__of_abortions_by_state_of_occurrence_2020'] * 1000) / df['No__of_abortions_per_1000_women_aged_15-44_by_state_of_occurrence_2020']

# Add state codes
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
df['State_Code'] = df['U_S__State'].map(state_abbreviations)
df = df.dropna(subset=['State_Code', 'Pop_15_44'])

# Add land area (square miles)
land_area = {
    'AL': 52420, 'AK': 665384, 'AZ': 113990, 'AR': 53179, 'CA': 163695, 'CO': 104094,
    'CT': 5543, 'DE': 2489, 'DC': 68, 'FL': 65758, 'GA': 59425, 'HI': 10932,
    'ID': 83569, 'IL': 57914, 'IN': 36420, 'IA': 56273, 'KS': 82278, 'KY': 40408,
    'LA': 52378, 'ME': 35380, 'MD': 12406, 'MA': 10554, 'MI': 96714, 'MN': 86936,
    'MS': 48432, 'MO': 69707, 'MT': 147040, 'NE': 77348, 'NV': 110572, 'NH': 9349,
    'NJ': 8723, 'NM': 121590, 'NY': 54555, 'NC': 53819, 'ND': 70698, 'OH': 44826,
    'OK': 69899, 'OR': 98379, 'PA': 46054, 'RI': 1545, 'SC': 32020, 'SD': 77116,
    'TN': 42144, 'TX': 268596, 'UT': 84897, 'VT': 9616, 'VA': 42775, 'WA': 71298,
    'WV': 24230, 'WI': 65496, 'WY': 97813
}
df['Land_Area'] = df['State_Code'].map(land_area)
df = df.dropna(subset=['Land_Area'])

# Normalize clinics and funding
df['Clinics_Score'] = df['No__of_abortion_clinics_2020'] / df['No__of_abortion_clinics_2020'].max()
df['Funding_Score'] = df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'] / df['Reported_public_expenditures_for_abortions_in_000s_of_dollars_state_2015'].max()
df['Raw_Resource_Score'] = df['Clinics_Score'] + df['Funding_Score']

# ✅ Adjust by population × land area
df['Adjusted_Resource_Score'] = df['Raw_Resource_Score'] / (df['Pop_15_44'] * df['Land_Area'])

# --------------------
# Plot 1: Raw Score (Unadjusted)
# --------------------
fig = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Raw_Resource_Score',
    color_continuous_scale=px.colors.sequential.Blues,
    scope='usa',
    title='California is Unmatched in Abortion Resources<br><sup>Total Clinics and State Abortion Funding — Without Adjusting for Population or Geography</sup>'
)
fig.update_layout(
    template="plotly_white",
    coloraxis_colorbar_title="Total Resource Score<br>(Clinics + Funding)",
    width=1150,
    height=550,
    margin=dict(l=60, r=60, t=90, b=40),
    title_x=0.5,
    title_font_size=28
)
fig.show()

# --------------------
# Plot 2: Adjusted Score (Per Capita × Land Area, Normalized Resources)
# --------------------
labels = ["0-20%", "20–40%", "40–60%", "60–80%", "80-100%"]
df['Density_Bin_Label'] = pd.qcut(df['Adjusted_Resource_Score'], q=5, labels=labels)
df['Density_Bin_Label'] = pd.Categorical(df['Density_Bin_Label'], categories=labels, ordered=True)

color_scale_discrete = ['#deebf7', '#9ecae1', '#6baed6', '#3182bd', '#08519c']

fig = px.choropleth(
    df,
    locations='State_Code',
    locationmode='USA-states',
    color='Density_Bin_Label',
    color_discrete_sequence=color_scale_discrete,
    category_orders={'Density_Bin_Label': labels},
    scope='usa',
    title='California is Falling Behind as East Coast Leads in Abortion Resources<br><sup>Based on Normalized Clinics and State Funding per Capita and per Square Mile</sup>'
)
fig.update_layout(
    template="plotly_white",
    legend_title="Percentile Based on:<br>(Normalized Clinics + Funding) / (Population × Land Area)",
    width=1150,
    height=550,
    margin=dict(l=60, r=60, t=90, b=40),
    title_x=0.5,
    title_font_size=28
)
fig.show()
